**Настройка параметров среды**

In [1]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.02 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.


In [1]:
# Изменить make-файл, чтобы сделать OpenCV и GPU доступными
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

[Errno 2] No such file or directory: 'darknet'
/content
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory


In [2]:
# Проверить версию CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
!make

make: *** No targets specified and no makefile found.  Stop.


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
# Сократите путь к Google Cloud Disk как mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Colab Notebooks'  'My Drive'   yolov4


**Обучение на собственных данных**

1 - Загрузка

In [8]:
# Текущий путь находится в даркнете
!ls
# Загрузить сжатый пакет изображения набора данных и помеченный txt файл на сервер
!cp /mydrive/yolov4/obj2.zip ../
# Распаковать в файл данных
!unzip ../obj2.zip -d data/

3rdparty	DarknetConfig.cmake.in	json_mjpeg_streams.sh  scripts
backup		darknet_images.py	LICENSE		       src
build		darknet.py		Makefile	       vcpkg.json
build.ps1	darknet_video.py	net_cam_v3.sh	       video_yolov3.sh
cfg		data			net_cam_v4.sh	       video_yolov4.sh
cmake		image_yolov3.sh		obj
CMakeLists.txt	image_yolov4.sh		README.md
darknet		include			results
Archive:  ../obj2.zip
   creating: data/obj2/
  inflating: data/obj2/example 245.jpg  
  inflating: data/obj2/example 245.txt  
  inflating: data/obj2/example 271.jpg  
  inflating: data/obj2/example 271.txt  
  inflating: data/obj2/example 272.jpg  
  inflating: data/obj2/example 272.txt  
  inflating: data/obj2/example 273.jpg  
  inflating: data/obj2/example 273.txt  
  inflating: data/obj2/example 274.jpg  
  inflating: data/obj2/example 274.txt  
  inflating: data/obj2/example 275.jpg  
  inflating: data/obj2/example 275.txt  
  inflating: data/obj2/example 276.jpg  
  inflating: data/obj2/example 276.txt  
  inflating: da

In [9]:
# Загрузите файл .cfg, измененный в соответствии с вашим собственным набором данных, из Google Cloud Disk
!cp /mydrive/yolov4/crane.cfg ./cfg
# Загрузите obj.data obj.names вашего собственного набора данных с Google Cloud Disk
!cp /mydrive/yolov4/crane.names ./data
!cp /mydrive/yolov4/crane.data  ./data
#Upload generate_train.py, чтобы сгенерировать train.txt на основе данных сервера
!cp /mydrive/yolov4/generate_train.py ./
# Запустить, чтобы сгенерировать train.txt под файлом данных
!python generate_train.py

In [ ]:
# скрипт для anchors
!./darknet detector calc_anchors data/crane.data -num_of_clusters 6 -width 416 -height 416

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0

 num_of_clusters = 6, width = 416, height = 416 
 read labels from 1003 images 
 loaded 	 image: 1003 	 box: 1375
 all loaded. 

 calculating k-means++ ...

 iterations = 24 


counters_per_class = 372, 1003

 avg IoU = 71.25 % 

Saving anchors to the file: anchors.txt 
anchors =  20, 31,  34, 98,  77, 76,  56,184, 149,163,  84,294
^C


In [10]:
# URL-адрес предтренировочного веса - это облачный диск с использованием метода загрузки
!cp /mydrive/yolov4/yolov4-tiny.conv.29 ./

In [ ]:
# обучение!
!./darknet detector train data/crane.data cfg/crane.cfg yolov4-tiny.conv.29 -dont_show -map

CUDA status Error: file: ./src/dark_cuda.c : () : line: 38 : build time: Mar 21 2022 - 15:11:24 

 CUDA Error: no CUDA-capable device is detected
Darknet error location: ./src/dark_cuda.c, check_error, line #69
CUDA Error: no CUDA-capable device is detected: Bad file descriptor


In [ ]:
# последние веса после checkpoint
!cp /mydrive/yolov4/crane_last.weights  ./data

cp: cannot stat '/mydrive/yolov4/crane_last.weights': No such file or directory


In [ ]:
# если слетело
!./darknet detector train data/crane.data cfg/crane.cfg data/crane_last.weights  -dont_show

Выходные данные были обрезаны до нескольких последних строк (5000).
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.872263), count: 2, class_loss = 0.000337, iou_loss = 2.935905, total_loss = 2.936242 
 total_bbox = 203869, rewritten_bbox = 0.057880 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.872958), count: 4, class_loss = 0.078813, iou_loss = 0.647002, total_loss = 0.725816 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.680287), count: 4, class_loss = 0.315111, iou_loss = 3.794153, total_loss = 4.109263 
 total_bbox = 203877, rewritten_bbox = 0.057878 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.928348), count: 4, class_loss = 0.000017, iou_loss = 0.717515, total_loss = 0.717532 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.910701), count: 1, class_loss = 0.000111, iou_loss = 0.241516, total_loss = 

In [11]:
# URL-адрес предтренировочного веса - это облачный диск с использованием метода загрузки
!cp /mydrive/yolov4/crane_final.weights ./

In [12]:
# видео для теста
!cp /mydrive/yolov4/task2.mp4 ./

In [13]:
!./darknet detector demo data/crane.data cfg/crane.cfg crane_final.weights -dont_show task2.mp4 -i 0 -out_filename new_task2.mp4

Выходные данные были обрезаны до нескольких последних строк (5000).
crane: 68% 

FPS:63.5 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 74% 
crane: 69% 

FPS:63.7 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 73% 
crane: 68% 

FPS:64.3 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 76% 
crane: 63% 

FPS:65.0 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 76% 
crane: 63% 

FPS:65.1 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 77% 
crane: 63% 

FPS:65.4 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 76% 
crane: 63% 

FPS:64.8 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 77% 
crane: 63% 

FPS:65.2 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 74% 
crane: 59% 

FPS:63.1 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 75% 
crane: 60% 

FPS:62.7 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 75% 
crane: 60% 

FPS:63.0 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 74% 
crane: 60% 

FPS:64.0 	 AVG_FPS:75.3

 cvWriteFrame 
Objects:

cart: 74% 
crane: 60% 

FPS:64.3 	 AVG_FPS:75.3

 cvWri

In [ ]:
# функция для скачивания
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# download the video with detections shown
download('new_task2.ts')